In [1]:
import pandas, warnings

warnings.filterwarnings("ignore")

In [2]:
rating = pandas.read_table("ratings.csv", sep=",")

rating.iloc[:5]

,UserID,MovieID,Rating,Timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [3]:
rating.tail()

,UserID,MovieID,Rating,Timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [4]:
null = rating.isnull().sum()

null.sum()

0

In [5]:
total_user = rating["UserID"].nunique()

total_user

610

In [6]:
total_movie = rating["MovieID"].nunique()

total_movie

9724

In [7]:
rata_rating = rating["Rating"].mean().round(3)

rata_rating

3.502

Average Rating of Our Rate Table is 3.5

In [8]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate = lambda table : round((table.isnull().sum().sum() / table.size) * 100, 3)

SparseRate(rating_table)

98.3

98.3 Sparsity Rate Mean that Our Table is Mostly Consist of 0 Value

Or Maybe, One or Two Movie is Only Watch Below 10 or User Only Watch Less Than 50 Movie

Other Reason is After Watch Movie, Our User Didn't Rate the Movies

In [9]:
minimum_film_interaction = 10

WatchFilm = rating.groupby("MovieID")["Rating"].agg(["count", "mean"])

WatchFilm.columns = ["TotalWatch", "Rating"]

WatchFilm = WatchFilm[WatchFilm["TotalWatch"] > minimum_film_interaction]

rating = rating[rating["MovieID"].isin(WatchFilm.index)]

rating["MovieID"].nunique()

2121

In [10]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate(rating_table)

93.845

In [11]:
minimum_user_interaction = 50

UserWatch = rating.groupby("UserID")[["Rating"]].count()

UserWatch = UserWatch[UserWatch["Rating"] > minimum_user_interaction]

rating = rating[rating["UserID"].isin(UserWatch.index)]

In [12]:
rating_table = rating.pivot_table(index="MovieID", columns="UserID", values="Rating")

SparseRate(rating_table)

90.396

After Filtering, Our Final Sparsity Rate is 90.396

In [13]:
rating_table.iloc[:5, :8]

UserID,1,4,6,7,10,11,15,16
MovieID,,,,,,,,
1,4.0,NaN,NaN,4.5,NaN,NaN,2.5,NaN
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN
6,4.0,NaN,4.0,NaN,NaN,5.0,NaN,NaN


In [14]:
rating_table = rating_table.fillna(0)

rating_table.iloc[:5, :8]

UserID,1,4,6,7,10,11,15,16
MovieID,,,,,,,,
1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0


In [15]:
rating_table.index.name

'MovieID'

In [16]:
len(rating_table.index)

2121

In [17]:
rating_table.columns.name

'UserID'

In [18]:
len(rating_table.columns)

352

What is Sparse Matrix ? Sparse Matrix is Matrix that Many of the Value is 0

Usually Sparse Matrix is Mostly Use in Collaborative Method Recommendation System

In [19]:
from scipy.sparse import csr_matrix

rating_matrix = csr_matrix(rating_table.values)

rating_matrix.shape

(2121, 352)

In [20]:
rating_matrix.toarray()[:5, :8]

array([[4. , 0. , 0. , 4.5, 0. , 0. , 2.5, 0. ],
       [0. , 0. , 4. , 0. , 0. , 0. , 0. , 0. ],
       [4. , 0. , 5. , 0. , 0. , 0. , 0. , 0. ],
       [0. , 0. , 5. , 0. , 0. , 0. , 0. , 0. ],
       [4. , 0. , 4. , 0. , 0. , 5. , 0. , 0. ]])

In [21]:
rating_matrix.toarray()[0, :10]

array([4. , 0. , 0. , 4.5, 0. , 0. , 2.5, 0. , 4.5, 3.5])

In [22]:
rating_table.iloc[0, :10].values

array([4. , 0. , 0. , 4.5, 0. , 0. , 2.5, 0. , 4.5, 3.5])

In [23]:
rating_reset = rating_table.reset_index()

rating_reset.shape

(2121, 353)

In [24]:
rating_table.iloc[:10]

UserID,1,4,6,7,10,11,15,16,17,18,...,600,601,602,603,604,605,606,607,608,610
MovieID,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,3.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0
7,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
11,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,2.5,3.0,0.0,0.0


In [25]:
rating_reset.iloc[:10]

UserID,MovieID,1,4,6,7,10,11,15,16,17,...,600,601,602,603,604,605,606,607,608,610
0,1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,...,2.5,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,5.0
1,2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0
2,3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
3,5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.5,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
4,6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,3.0,4.0,3.0,0.0,0.0,0.0,0.0,5.0
5,7,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0
6,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,10,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
8,11,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,2.5,3.0,0.0,0.0
9,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
len(rating_reset.index)

2121

In [ ]:
from sklearn.neighbors import NearestNeighbors

model = NearestNeighbors(metric="cosine", algorithm="brute")

model.fit(rating_matrix)

In [28]:
usecols = ["MovieID", "Title"]

table_film = pandas.read_table("movies.csv", sep=",", usecols=usecols)

table_film.iloc[:5]

,MovieID,Title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [29]:
rating = rating.join(table_film.set_index("MovieID"), on="MovieID")

rating.iloc[:5]

,UserID,MovieID,Rating,Timestamp,Title
0,1,1,4.0,964982703,Toy Story (1995)
1,1,3,4.0,964981247,Grumpier Old Men (1995)
2,1,6,4.0,964982224,Heat (1995)
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995)
4,1,50,5.0,964982931,"Usual Suspects, The (1995)"


In [30]:
movies_index = table_film[table_film["Title"].str.contains("Avengers")]

movies_index.iloc[:5]

,MovieID,Title
1611,2153,"Avengers, The (1998)"
6148,44020,Ultimate Avengers (2006)
7693,89745,"Avengers, The (2012)"
8551,115727,Crippled Avengers (Can que) (Return of the 5 D...
8686,122892,Avengers: Age of Ultron (2015)


In [31]:
movies_index = movies_index.iloc[2]['MovieID']

movies_index

89745

In [32]:
total_limit = 10

title = table_film[table_film["MovieID"] == movies_index]["Title"]

title.values[0]

'Avengers, The (2012)'

In [33]:
rating[rating["MovieID"] == movies_index].iloc[:5]

,UserID,MovieID,Rating,Timestamp,Title
1535,15,89745,2.0,1510572842,"Avengers, The (2012)"
2182,18,89745,4.0,1455050365,"Avengers, The (2012)"
3520,21,89745,4.0,1418063440,"Avengers, The (2012)"
7891,52,89745,5.0,1468051351,"Avengers, The (2012)"
9030,62,89745,4.0,1521488914,"Avengers, The (2012)"


In [34]:
rating_reset.iloc[:5, :10]

UserID,MovieID,1,4,6,7,10,11,15,16,17
0,1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5
1,2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
4,6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0


In [35]:
movies_index = rating_reset[rating_reset["MovieID"] == movies_index]

movies_index

UserID,MovieID,1,4,6,7,10,11,15,16,17,...,600,601,602,603,604,605,606,607,608,610
1961,89745,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


In [36]:
movies_index = movies_index.index[0]

movies_index

1961

In [37]:
total_limit += 1

distances, indices = model.kneighbors(rating_matrix[movies_index], n_neighbors=total_limit)

type(distances), type(indices)

(numpy.ndarray, numpy.ndarray)

In [38]:
sorter = list(zip(indices.flatten().tolist(), distances.flatten().tolist()))

indices = sorted(sorter, key=lambda x: x[1])

indices[:5]

[(1961, 0.0),
 (2060, 0.23810542775945065),
 (1821, 0.2771270484831855),
 (1907, 0.29182116047789386),
 (1970, 0.2938930732962426)]

In [39]:
rating_reset[rating_reset.index == 2060]

UserID,MovieID,1,4,6,7,10,11,15,16,17,...,600,601,602,603,604,605,606,607,608,610
2060,112852,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,...,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5


In [40]:
rating_reset.iloc[2060, 0]

112852

In [41]:
rating[rating["MovieID"] == rating_reset.iloc[2060, 0]].iloc[:5]

,UserID,MovieID,Rating,Timestamp,Title
1551,15,112852,1.5,1510571973,Guardians of the Galaxy (2014)
2225,18,112852,4.5,1455618433,Guardians of the Galaxy (2014)
3602,21,112852,4.0,1412967449,Guardians of the Galaxy (2014)
7901,52,112852,5.0,1468051607,Guardians of the Galaxy (2014)
9081,62,112852,4.0,1521489638,Guardians of the Galaxy (2014)


In [42]:
molist = [i[0] for i in indices]

molist = [i for i in rating_reset.iloc[molist, 0].values][1:]

molist[:5]

[112852, 59315, 77561, 91529, 111362]

In [43]:
result = table_film[table_film["MovieID"].isin(molist)]

result

,MovieID,Title
6743,59315,Iron Man (2008)
7324,77561,Iron Man 2 (2010)
7372,79132,Inception (2010)
7620,87232,X-Men: First Class (2011)
7768,91529,"Dark Knight Rises, The (2012)"
8151,102125,Iron Man 3 (2013)
8395,110102,Captain America: The Winter Soldier (2014)
8425,111362,X-Men: Days of Future Past (2014)
8475,112852,Guardians of the Galaxy (2014)
8683,122886,Star Wars: Episode VII - The Force Awakens (2015)


In [44]:
def OutputFilm(title, limit=5):

  movies_index = table_film[table_film["Title"] == title]

  movies_index = movies_index["MovieID"].values[0]

  movies_index = rating_reset[rating_reset["MovieID"] == movies_index].index[0]

  limit += 1

  distances, indices = model.kneighbors(rating_matrix[movies_index], n_neighbors=limit)

  sorter = list(zip(indices.flatten().tolist(), distances.flatten().tolist()))

  indices = sorted(sorter, key=lambda x: x[1])

  molist = [i[0] for i in indices]

  molist = [i for i in rating_reset.iloc[molist, 0].values][1:]

  result = table_film[table_film["MovieID"].isin(molist)]

  return result

OutputFilm("Avengers, The (2012)", 15)

,MovieID,Title
6743,59315,Iron Man (2008)
7214,73017,Sherlock Holmes (2009)
7324,77561,Iron Man 2 (2010)
7372,79132,Inception (2010)
7589,86332,Thor (2011)
7620,87232,X-Men: First Class (2011)
7646,88140,Captain America: The First Avenger (2011)
7768,91529,"Dark Knight Rises, The (2012)"
8053,98809,"Hobbit: An Unexpected Journey, The (2012)"
8151,102125,Iron Man 3 (2013)


In [45]:
OutputFilm(table_film[table_film["MovieID"] == 2]["Title"].values[0], 10)

,MovieID,Title
131,158,Casper (1995)
275,316,Stargate (1994)
322,364,"Lion King, The (1994)"
325,367,"Mask, The (1994)"
418,480,Jurassic Park (1993)
436,500,Mrs. Doubtfire (1993)
483,551,"Nightmare Before Christmas, The (1993)"
504,586,Home Alone (1990)
506,588,Aladdin (1992)
512,595,Beauty and the Beast (1991)


In [46]:
OutputFilm("Forrest Gump (1994)")

,MovieID,Title
97,110,Braveheart (1995)
257,296,Pulp Fiction (1994)
277,318,"Shawshank Redemption, The (1994)"
418,480,Jurassic Park (1993)
1939,2571,"Matrix, The (1999)"


In [47]:
rating_table.iloc[:10, :10]

UserID,1,4,6,7,10,11,15,16,17,18
MovieID,,,,,,,,,,
1,4.0,0.0,0.0,4.5,0.0,0.0,2.5,0.0,4.5,3.5
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0
7,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.0,0.0,3.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
11,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
